# Scraping Researcher Ages on Veromi

## Python Setup

In [2]:
from multiprocessing import Pool
import multiprocessing
import pandas as pd
import requests
from bs4 import BeautifulSoup
import unidecode
from veromi_query import query

## Load In Individuals

In [3]:
ind = pd.read_csv('../data/ind_info.csv', low_memory=False)

In [4]:
del ind['Unnamed: 0'], ind['selected_subset'], ind['last_name']
ind.rename(columns={'full_last_name': 'last_name'}, inplace = True)

In [5]:
ind.describe(include='all')

,inventor_id,first_name,middle_name,last_name,city,state,country
count,4980929,4979455,2223368,4979597,4970991,2264690,4975800
unique,3663409,194144,49122,631580,103621,61,249
top,4090850-1,John,J.,Lee,Tokyo,CA,US
freq,200,94614,169824,37219,124683,447839,2266802


In [6]:
ind = ind[ind['country']=="US"].reset_index(drop=True)

In [7]:
ind.describe(include='all')

,inventor_id,first_name,middle_name,last_name,city,state,country
count,2266802,2266745,1637819,2266783,2265349,2264690,2266802
unique,1796830,86199,28147,357823,22141,61,1
top,5766379-2,John,J.,Smith,San Jose,CA,US
freq,67,78303,142608,12895,33910,447839,2266802


## Data Cleanup

In [8]:
string_vars = ['first_name', 'middle_name', 'last_name', 'city', 'state', 'country']
for var in string_vars:
    ind[var] = ind[var].str.upper().str.strip().fillna('')

Let's create an ID variable for every Individual\*Location combination.

In [9]:
ind['id'] = ind.index

In [10]:
ind.head()

,inventor_id,first_name,middle_name,last_name,city,state,country,id
0,3930274-1,HAROLD,A.,SYFRITT,SAN PEDRO,CA,US,0
1,3930274-1,HAROLD,A.,SYFRITT,MANHATTAN BEACH,CA,US,1
2,3930275-1,RICHARD,,BAILIN,ENGLEWOOD CLIFFS,NJ,US,2
3,3930276-1,RUSSEL,A.,VAN BRAKEL,HARSHAW,WI,US,3
4,3930276-1,RUSSEL,A.,VAN BRAKEL,ONEIDA,WI,US,4


In [11]:
ind.to_csv('../data/ind_clean.csv', index=False)

In [12]:
ind.head(10000).to_csv('../data/ind_clean_sample.csv', index=False)

## Scrape Profiles

__Run `age_scraper.py` code.__

The following code works for a limited number of queries.

```
import pandas as pd
from veromi_query import query

# Load in data:
ind = pd.read_csv('../data/ind_clean.csv', low_memory=False)
ind = ind.fillna('')

# Query Veromi:
profiles = query([[0, ind, ind.shape[0]])

# Export to CSV:
profiles.to_csv('../data/profiles.csv', index=False)
```

When running a large number of queries, I adapt the code for parallelization. Here is the example for 3 parallel tasks. In reality, I used a clustered computing system and 100 parallel tasks.

```
from multiprocessing import Pool
import multiprocessing
import pandas as pd
from veromi_query import query


# Load in data:
ind = pd.read_csv('../data/ind_clean.csv', low_memory=False)
ind = ind.fillna('')

# Define Pool:
pool = Pool(processes=3)

# Run query for the job:
profiles0 = pool.apply_async(query, [[0, ind, 1000000]])
profiles1 = pool.apply_async(query, [[1, ind, 1000000]])
profiles2 = pool.apply_async(query, [[2, ind, 1000000]])
profiles0 = profiles0.get()
profiles1 = profiles1.get()
profiles2 = profiles2.get()

# Combine Frames:
frames = [profiles0, profiles1, profiles2]

profiles = pd.concat(frames).reset_index(drop=True)

# Export results as CSV
profiles.to_csv('../data/profiles_ind_clean.csv', index=False)
```

## Parse Age from Data

In [13]:
nb_processes = 100

In [104]:
# Load in entire dataset
profiles = pd.read_csv('../output/profiles_ind_clean.csv')

In [105]:
profiles.head()

,id,name
0,0.0,"SYFRITT, HAROLD ARTHUR (Age 79)"
1,1.0,"SYFRITT, HAROLD ARTHUR (Age 79)"
2,2.0,"BAILIN, RICHARD C (Age 91)"
3,2.0,"BAILIN, RICHARD"
4,3.0,"VANBRAKEL, RUSSEL A (Age 81)"


In [106]:
profiles['age'] = profiles['name'].str.extract(r'(\(Age\s\d+\))', expand=True)
profiles['age'] = pd.to_numeric(profiles['age'].str.extract(r'(\d+)', expand=False))

In [107]:
profiles.head()

,id,name,age
0,0.0,"SYFRITT, HAROLD ARTHUR (Age 79)",79.0
1,1.0,"SYFRITT, HAROLD ARTHUR (Age 79)",79.0
2,2.0,"BAILIN, RICHARD C (Age 91)",91.0
3,2.0,"BAILIN, RICHARD",NaN
4,3.0,"VANBRAKEL, RUSSEL A (Age 81)",81.0


For now, let's keep the most common age associated to a given Individual\*Location ID. This also gets rid on `NaN` values.

In [108]:
age_lookup = profiles.groupby(['id', 'age']).size().reset_index().rename(columns={0: 'match_count'})
age_lookup = age_lookup.sort_values(['id', 'match_count'], ascending=[True, False])
age_lookup = age_lookup.drop_duplicates('id', keep='first')

## Merge back onto the Original Data of Individuals\*Locations

In [109]:
ind = pd.read_csv('../data/ind_clean.csv')

In [110]:
ind = pd.merge(ind, age_lookup, how = 'left', on = 'id')

In [111]:
ind.describe(include='all')

,inventor_id,first_name,middle_name,last_name,city,state,country,id,age,match_count
count,2266802,2266711,1637791,2266688,2265349,2264690,2266802,2266802.0,1.876797e+06,1.876797e+06
unique,1796830,85489,27940,355369,22125,61,1,2266802.0,NaN,NaN
top,5766379-2,JOHN,J.,SMITH,SAN JOSE,CA,US,2266801.0,NaN,NaN
freq,67,78303,142609,12895,33910,447839,2266802,1.0,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.974880e+01,1.299313e+00
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.510506e+01,7.314095e-01
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.800000e+01,1.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.900000e+01,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.800000e+01,1.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.900000e+01,1.000000e+00


In [112]:
ind.head(20)

,inventor_id,first_name,middle_name,last_name,city,state,country,id,age,match_count
0,3930274-1,HAROLD,A.,SYFRITT,SAN PEDRO,CA,US,0,79.0,1.0
1,3930274-1,HAROLD,A.,SYFRITT,MANHATTAN BEACH,CA,US,1,79.0,1.0
2,3930275-1,RICHARD,NaN,BAILIN,ENGLEWOOD CLIFFS,NJ,US,2,91.0,1.0
3,3930276-1,RUSSEL,A.,VAN BRAKEL,HARSHAW,WI,US,3,81.0,1.0
4,3930276-1,RUSSEL,A.,VAN BRAKEL,ONEIDA,WI,US,4,81.0,1.0
5,3930277-1,RICHARD,F.,WULFF,MAPLE GROOVE,MN,US,5,80.0,2.0
6,3930277-1,RICHARD,F.,WULFF,MAPLE PLAIN,MN,US,6,80.0,1.0
7,3930277-1,RICHARD,F.,WULFF,LONG LAKE,MN,US,7,80.0,2.0
8,3930278-1,RICHARD,A.,NASCA,PAINESVILLE,OH,US,8,87.0,3.0
9,3930280-1,MARTIN,M.,VASAS,FAIRFIELD,CT,US,9,53.0,2.0


In [113]:
ind.tail(20)

,inventor_id,first_name,middle_name,last_name,city,state,country,id,age,match_count
2266782,T990007-1,GURDIP,S.,SETHI,ROCHESTER,NY,US,2266782,73.0,2.0
2266783,T991001-1,LEONARD,D.,DONOVAN,SOUTHBOROUGH,MA,US,2266783,NaN,NaN
2266784,T991001-2,MATTHEW,M.,SITTER,CONVENT STATION,NJ,US,2266784,92.0,1.0
2266785,T993002-4,JAN,M.,POTTINGER,FLORENCE,AL,US,2266785,NaN,NaN
2266786,T994001-2,CHARLES,W.,DYSON,LOS ANGELES,CA,US,2266786,73.0,1.0
2266787,T994002-2,NORMAN,E.,HESS,ROCHESTER,NY,US,2266787,47.0,1.0
2266788,T995001-1,STEPHEN,NaN,BASS,WILMINGTON,DE,US,2266788,94.0,1.0
2266789,T995003-1,VICTOR,C.,ZIMMERMAN,ST. LOUIS,MO,US,2266789,29.0,1.0
2266790,T995005-1,KENNETH,J.,MERKEL,BROOKFIELD,WI,US,2266790,64.0,1.0
2266791,T995005-2,LOUIS,T.,SVITKOVICH,MILWAUKEE,WI,US,2266791,71.0,1.0


Notes to self:
- Remove Jr.

### Assign Unique Age to Individual ID

In [114]:
ind_id_age = ind.groupby(['inventor_id', 'age'])['match_count'].sum().reset_index()
ind_id_age = ind_id_age.sort_values(['inventor_id', 'match_count'], ascending=[True, False])
ind_id_age = ind_id_age.drop_duplicates(['inventor_id'], keep='first')

# Merge to unique list of Individual IDs
unique_inds = ind[['inventor_id', 'first_name', 'middle_name', 'last_name']].drop_duplicates()
ind_id_age = pd.merge(unique_inds, ind_id_age, how='left', on='inventor_id')

In [115]:
ind_id_age.head()

,inventor_id,first_name,middle_name,last_name,age,match_count
0,3930274-1,HAROLD,A.,SYFRITT,79.0,2.0
1,3930275-1,RICHARD,NaN,BAILIN,91.0,1.0
2,3930276-1,RUSSEL,A.,VAN BRAKEL,81.0,2.0
3,3930277-1,RICHARD,F.,WULFF,80.0,5.0
4,3930278-1,RICHARD,A.,NASCA,87.0,3.0


In [116]:
ind_id_age.describe(include='all')

,inventor_id,first_name,middle_name,last_name,age,match_count
count,1796830,1796754,1278672,1796733,1.494904e+06,1.494904e+06
unique,1796830,85489,27940,355369,NaN,NaN
top,7939156-2,JOHN,J.,SMITH,NaN,NaN
freq,1,59092,110732,8210,NaN,NaN
mean,NaN,NaN,NaN,NaN,6.002239e+01,1.497659e+00
std,NaN,NaN,NaN,NaN,1.535876e+01,1.015337e+00
min,NaN,NaN,NaN,NaN,1.800000e+01,1.000000e+00
25%,NaN,NaN,NaN,NaN,4.900000e+01,1.000000e+00
50%,NaN,NaN,NaN,NaN,5.800000e+01,1.000000e+00
75%,NaN,NaN,NaN,NaN,6.900000e+01,2.000000e+00


In [128]:
inv_id_age.to_csv('../output/inv_id_age.csv', index=False)

## Sandbox